<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/EstrattoreFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Riccardo VEnturi Cagliari 2025
#Questa è una funzione che puoi scrivere e testare in PyCharm, VSCode, o qualsiasi altro editor. Non ha bisogno di GPU o di Colab. Prende una maschera e sputa fuori numeri. Semplice, pulito, gratificante.
import cv2
import numpy as np
from pathlib import Path


def extract_features_from_mask(mask_path: Path):
    """
    Carica una maschera di predizione e calcola un set di feature quantitative.

    Args:
        mask_path (Path): Il percorso del file della maschera (es. "..._pred.png").

    Returns:
        dict: Un dizionario contenente le feature calcolate, o None se la maschera è invalida.
    """
    # --- 1. CARICAMENTO E PREPARAZIONE ---
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Errore: impossibile leggere la maschera {mask_path}")
        return None

    # Isola le singole classi in maschere binarie separate (0 o 1)
    foro_mask = (mask == 1).astype(np.uint8)
    danno_mask = (mask == 2).astype(np.uint8)

    # Inizializza il dizionario delle feature con valori di default
    features = {
        # Feature del Foro
        'area_foro_px': 0,
        'diametro_foro_px': 0,

        # Feature del Danno
        'area_danno_totale_px': 0,
        'numero_blob_danno': 0,
        'rapporto_danno_foro': 0.0,  # Metrica chiave, adimensionale

        # Feature di Forma del Danno (aggregate)
        'eccentricita_media_danno': 0.0,
        'solidita_media_danno': 0.0,
        'area_danno_piu_grande': 0.0,
    }

    # --- 2. ANALISI DEL FORO ---
    contours_foro, _ = cv2.findContours(foro_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours_foro:
        # Assumiamo che il contorno più grande sia il foro
        main_foro_contour = max(contours_foro, key=cv2.contourArea)
        area_foro = cv2.contourArea(main_foro_contour)
        if area_foro > 0:
            features['area_foro_px'] = area_foro
            # Diametro di un cerchio con la stessa area
            features['diametro_foro_px'] = np.sqrt(4 * area_foro / np.pi)

    # --- 3. ANALISI DEL DANNO (BLOB PER BLOB) ---
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(danno_mask, connectivity=8)

    # num_labels include lo sfondo (label 0), quindi il numero di blob è num_labels - 1
    num_blob_danno = num_labels - 1
    features['numero_blob_danno'] = num_blob_danno

    if num_blob_danno > 0:
        # Estrai le aree di tutti i blob di danno (escluso lo sfondo)
        aree_danno = stats[1:, cv2.CC_STAT_AREA]
        features['area_danno_totale_px'] = int(np.sum(aree_danno))
        features['area_danno_piu_grande'] = int(np.max(aree_danno))

        # Calcola le feature di forma per ogni blob e fanne la media
        eccentricities = []
        solidities = []
        for i in range(1, num_labels):
            blob_mask = (labels == i).astype(np.uint8)
            contours_danno, _ = cv2.findContours(blob_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours_danno: continue

            cnt = contours_danno[0]
            # Solidità: rapporto tra area del contorno e area del suo inviluppo convesso
            area_cnt = cv2.contourArea(cnt)
            hull = cv2.convexHull(cnt)
            hull_area = cv2.contourArea(hull)
            if hull_area > 0:
                solidities.append(area_cnt / hull_area)

            # Eccentricità: richiede di fittare un'ellisse
            if len(cnt) >= 5:  # fitEllipse richiede almeno 5 punti
                _, (MA, ma), _ = cv2.fitEllipse(cnt)
                eccentricities.append(np.sqrt(1 - (ma / MA) ** 2))

        if eccentricities:
            features['eccentricita_media_danno'] = np.mean(eccentricities)
        if solidities:
            features['solidita_media_danno'] = np.mean(solidities)

    # --- 4. CALCOLO FEATURE COMBINATE ---
    if features['area_foro_px'] > 0:
        features['rapporto_danno_foro'] = features['area_danno_totale_px'] / features['area_foro_px']

    return features
# --- ESEMPIO DI UTILIZZO ---
if __name__ == '__main__':
    # Crea una maschera finta per il test (o usa una delle tue maschere predette)
    # Assicurati che il percorso sia corretto
    test_mask_path = Path("/home/ricc/Downloads/MAsk_patches/H064_h064_Carbon Textile 1A.png") # Esempio

    if test_mask_path.exists():
        extracted_features = extract_features_from_mask(test_mask_path)
        if extracted_features:
            print(f"--- Feature estratte da {test_mask_path.name} ---")
            for key, value in extracted_features.items():
                print(f"{key:<30}: {value:.4f}")
    else:
        print(f"File di test non trovato: {test_mask_path}. Impossibile eseguire l'esempio.")


In [ ]:
Ecco una classifica, dalla più semplice alla più potente:

**Livello 1: Features Basilari (Grandezza e Posizione)**
*   `class_id`: 1 per "foro", 2 per "danno". Fondamentale.
*   `area`: L'area in pixel del difetto (`m00` dei momenti). La feature più importante in assoluto.
*   `centroid_x`, `centroid_y`: Le coordinate del baricentro (`m10/m00`, `m01/m00`). Utili per capire la posizione del difetto sulla patch.
*   `perimeter`: La lunghezza del contorno. Utile per calcolare la "compattezza".

**Livello 2: Descrittori di Forma Semplici (Quanto è "strano"?)**
*   `eccentricity`: **Fondamentale.** Misura quanto un oggetto è "allungato" come un'ellisse. Varia da 0 (cerchio perfetto) a 1 (linea retta). Una crepa avrà eccentricità vicina a 1, un foro vicino a 0.
*   `solidity`: **Fondamentale.** Rapporto tra l'area del contorno e l'area del suo inviluppo convesso (pensa a un elastico teso attorno alla forma). Una forma liscia e convessa ha solidità 1. Una forma "frastagliata" o a forma di 'C' ha solidità bassa. Distingue una macchia corrosiva da un foro netto.
*   `extent`: Rapporto tra l'area del contorno e l'area del rettangolo che lo contiene. Simile alla solidità, ma sensibile all'orientamento.
*   `aspect_ratio`: Rapporto tra larghezza e altezza del rettangolo contenitore. Simile all'eccentricità ma più semplice.

**Livello 3: Momenti di Hu (Il DNA della Forma)**
*   `hu_0` a `hu_6`: Questi sono i tuoi 7 assi nella manica. Sono **invarianti** a traslazione, scala e rotazione. Questo significa che un piccolo danno a forma di "L" e un grande danno a forma di "L" ruotato avranno momenti di Hu molto simili.
    *   **Come interpretarli (in modo intuitivo):**
        *   `hu_0`, `hu_1`: Legati all'inerzia e all'allungamento (simili ad area ed eccentricità ma più stabili).
        *   `hu_2`, `hu_3`: Catturano le asimmetrie (skewness). Distinguono una 'T' da una 'L'.
        *   `hu_4`, `hu_5`, `hu_6`: Catturano dettagli ancora più sottili, concavità, "punte". `hu_6` è sensibile alla torsione (distingue una 'S' dalla sua immagine speculare).
    *   **IMPORTANTE:** Come hai già scoperto, i valori dei momenti di Hu possono essere piccolissimi o grandissimi. La trasformazione `log` che hai usato (`np.sign(hu)*np.log10(np.abs(hu))`) è **ESSENZIALE** per renderli utilizzabili da un modello di machine learning.

In [ ]:
# =============================================================================
# CELLA PER ESTRAZIONE FEATURES AVANZATA (DA USARE SU COLAB)
# =============================================================================
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

def calculate_eccentricity(moments):
    """Calcola l'eccentricità dagli autovalori della matrice di covarianza."""
    # Matrice di covarianza dei momenti centralizzati
    mu20 = moments['mu20']
    mu02 = moments['mu02']
    mu11 = moments['mu11']

    # Calcolo degli autovalori
    # Questi rappresentano la varianza lungo gli assi principali
    term1 = (mu20 + mu02) / 2
    term2 = np.sqrt(4 * mu11**2 + (mu20 - mu02)**2) / 2

    lambda1 = term1 + term2  # Autovalore maggiore (semi-asse maggiore al quadrato)
    lambda2 = term1 - term2  # Autovalore minore (semi-asse minore al quadrato)

    # Evita divisione per zero o radici di negativi
    if lambda1 <= 0 or lambda2 < 0:
        return 0.0

    # L'eccentricità è sqrt(1 - (lambda_min / lambda_max))
    ecc = np.sqrt(1 - lambda2 / lambda1)
    return ecc

def extract_all_features_from_mask(mask, image_id="unknown"):
    """
    Estrae un set completo di features per OGNI contour trovato in una maschera.
    Ritorna una lista di dizionari, dove ogni dizionario è un difetto.
    """
    if mask is None:
        return []

    # Trova tutti i contorni, mantenendo la gerarchia
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

    if not contours:
        return []

    features_list = []

    for i, cnt in enumerate(contours):
        # Ignora i contorni troppo piccoli, spesso sono solo rumore
        if cv2.contourArea(cnt) < 5: # Soglia minima di 5 pixel, puoi aggiustarla
            continue

        # Estrai la classe del difetto. Prendi un punto qualsiasi del contorno.
        class_id = int(mask[cnt[0][0][1], cnt[0][0][0]])
        if class_id == 0: # Ignora lo sfondo
            continue

        # --- Calcolo Features ---
        mom = cv2.moments(cnt)

        # SALVAVITA: Se l'area è 0, salta il contorno per evitare errori NaN
        if mom['m00'] <= 0:
            continue

        # Livello 1: Features Basilari
        area = mom['m00']
        centroid_x = mom['m10'] / area
        centroid_y = mom['m01'] / area
        perimeter = cv2.arcLength(cnt, True)

        # Livello 2: Descrittori di Forma
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = float(w) / h if h != 0 else 0
        extent = area / (w * h) if (w * h) != 0 else 0

        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        solidity = area / hull_area if hull_area > 0 else 0

        eccentricity = calculate_eccentricity(mom)

        # Livello 3: Momenti di Hu (con la TUA normalizzazione logaritmica)
        hu = cv2.HuMoments(mom).flatten()
        hu_log = np.sign(hu) * np.log10(np.abs(hu) + 1e-30) # Aggiungo 1e-30 per evitare log(0)

        # Compila il dizionario delle features per QUESTO contorno
        features = {
            'image_id': image_id,
            'class_id': class_id,
            'area': area,
            'perimeter': perimeter,
            'centroid_x': centroid_x,
            'centroid_y': centroid_y,
            'eccentricity': eccentricity,
            'solidity': solidity,
            'extent': extent,
            'aspect_ratio': aspect_ratio,
            'hu_0': hu_log[0],
            'hu_1': hu_log[1],
            'hu_2': hu_log[2],
            'hu_3': hu_log[3],
            'hu_4': hu_log[4],
            'hu_5': hu_log[5],
            'hu_6': hu_log[6],
        }
        features_list.append(features)

    return features_list

In [ ]:
if __name__ == '__main__':
    # Imposta il percorso della cartella con le maschere del tuo algoritmo
    # Questo girerà su Colab, quindi monta il tuo Drive
    # from google.colab import drive
    # drive.mount('/content/drive')
    # MASK_DIR = Path("/content/drive/MyDrive/path/alle/tue/maschere_algoritmo")

    # Per un test locale, usa un percorso locale
    MASK_DIR = Path("/content/Masks") # Cambia questo percorso!
    MASK_DIR.mkdir(exist_ok=True) # Crea la cartella se non esiste
    print(f"ATTENZIONE: Stiamo usando la cartella di test '{MASK_DIR}'. Assicurati che contenga delle maschere .png")


    all_mask_paths = sorted(list(MASK_DIR.glob("*.png")))

    if not all_mask_paths:
        print("Nessuna maschera trovata nella cartella. L'esempio non può continuare.")
    else:
        print(f"Trovate {len(all_mask_paths)} maschere. Inizio estrazione...")

        all_features = []
        for mask_path in tqdm(all_mask_paths):
            mask_image = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
            features_from_one_mask = extract_all_features_from_mask(mask_image, image_id=mask_path.stem)
            all_features.extend(features_from_one_mask) # Aggiungi la lista di features alla lista principale

        # Converti tutto in un bellissimo DataFrame di Pandas
        features_df = pd.DataFrame(all_features)

        print("\n--- Estrazione Completata! ---")
        print(f"Estratte un totale di {len(features_df)} features da {len(all_mask_paths)} maschere.")
        print("\nPrime 5 righe del DataFrame risultante:")
        print(features_df.head())

        # Ora puoi fare l'analisi (EDA) o salvare il CSV per la LSTM
        output_csv_path = "extracted_features.csv"
        features_df.to_csv(output_csv_path, index=False)
        print(f"\nDataFrame salvato in '{output_csv_path}'")